Import scripts with necessary imports, models and train-test loop logic

In [1]:
from imports import*
from train_val_test import*
from models import*

import preprocessed data from external file

In [2]:
X_file = 'preprocessed_ppg512hz.npy'
y_file = 'y_ppg512hz.npy'
X_padded = np.load(X_file)
y = np.load(y_file)

In [3]:
# Convert the labels to binary (0 for classes 1 and 2, 1 for class 3)
y = np.array(y)
y_binary = np.where(y == 3, 1, 0)

# Train-validation-test split
X_train_val, X_test, y_train_val, y_test = train_test_split(X_padded, y_binary, test_size=0.4, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, train_size=0.5, random_state=42)

# Reshape the data to add a channel dimension for the CNN
X_train = X_train[..., np.newaxis].squeeze(-1)
X_val = X_val[..., np.newaxis].squeeze(-1)
X_test = X_test[..., np.newaxis].squeeze(-1)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

input_shape = (X_train_tensor.shape[0], X_train_tensor.shape[2])

X_train shape: (1533, 640, 1)
X_val shape: (1533, 640, 1)
X_test shape: (2045, 640, 1)


In [ ]:
# Training and evaluation loop
num_epochs = 200
patience = 50

criterion = nn.MSELoss()

best_model, best_hyperparams = grid_search(train_loader, val_loader, num_epochs, patience)

# Evaluate the best model on the test set
test_loss, test_accuracy, y_test_true, y_test_pred = evaluate_model(best_model, test_loader, criterion)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Prediction
y_pred_classes = (np.array(y_test_pred) > 0.5).astype(int).flatten()
y_test_classes = y_test_true

# Calculate additional metrics
precision = precision_score(y_test_classes, y_pred_classes)
recall = recall_score(y_test_classes, y_pred_classes)
f1 = f1_score(y_test_classes, y_pred_classes)
auc = roc_auc_score(y_test_classes, y_test_pred)
 
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1 Score: {:.2f}".format(f1))
print("AUC: {:.2f}".format(auc))

In [ ]:
pytorch_total_params = sum(p.numel() for p in best_model.parameters() if p.requires_grad)
print(pytorch_total_params)